### Setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
import geopandas as gpd

In [6]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [7]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

In [8]:
# spreadsheet_key = "1sCb1YbQ3-1oiL-cnK0yCkwFcQpvpf0efIngAUeC1ixo"
# book = gc.open_by_key(spreadsheet_key)

In [9]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

### Maternal death data
a lot of the data is suppressed, so could not break down by cause 

In [10]:
df = pd.read_table("data/raw/bystate_race_2020.txt", dtype={"State Code": str}).drop(
    columns=["Notes", "Race Code", "Hispanic Origin Code", "Crude Rate", "Population"]
)
df = df[~df["Deaths"].isna()]

In [11]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [12]:
values = [
    ((df["race"] == "White") & (df["hispanic_origin"] == "Not Hispanic or Latino")),
    (
        (df["race"] == "Black or African American")
        & (df["hispanic_origin"] == "Not Hispanic or Latino")
    ),
]

conditions = [
    "White",
    "Black",
]

df["race_whispanicorigin"] = np.select(values, conditions)

In [13]:
df.groupby("race_whispanicorigin").agg({"deaths": sum}).reset_index()

# You can also do it like this:

# cities = (
#     df.groupby(["city"]).agg({'count': sum}).reset_index()
# )

,race_whispanicorigin,deaths
0,0,84.0
1,Black,185.0
2,White,196.0


In [14]:
84 + 185 + 196

465

so looking at it by state we're missing a lot of deaths 
but let's look at the map anyways

In [15]:
deaths = (
    df.groupby(["state", "state_code", "race_whispanicorigin"])
    .agg({"deaths": sum})
    .reset_index()
)

### Natality data

In [16]:
natality_df = pd.read_table(
    "data/raw/natality_state_race_2020.txt", dtype={"State Code": str}
).drop(columns=["Mother's Hispanic Origin Code", "Mother's Single Race Code"])

In [17]:
natality_df.columns = (
    natality_df.columns.str.lower().str.replace(" ", "_").str.replace("'", "")
)

In [18]:
aapi = ["Native Hawaiian or Other Pacific Islander", "Asian"]

birth_conditions = [
    (
        (natality_df["mothers_single_race"] == "White")
        & (natality_df["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_df["notes"] != "Total")
    ),
    (
        (natality_df["mothers_single_race"] == "Black or African American")
        & (natality_df["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_df["notes"] != "Total")
    ),
    (
        (natality_df["mothers_single_race"].isin(aapi))
        & (natality_df["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_df["notes"] != "Total")
    ),
    (
        (natality_df["mothers_single_race"] == "American Indian or Alaska Native")
        & (natality_df["mothers_hispanic_origin"] == "Not Hispanic or Latino")
        & (natality_df["notes"] != "Total")
    ),
    (
        (natality_df["mothers_hispanic_origin"] == "Hispanic or Latino")
        & (natality_df["notes"] != "Total")
    ),
]

birth_values = [
    "White",
    "Black",
    "AAPI",
    "American Indian or Alaska Native",
    "Hispanic or Latino",
]

natality_df["race_whispanicorigin"] = np.select(birth_conditions, birth_values)

In [19]:
births = (
    natality_df.drop(columns=["notes"])
    .groupby(["state", "state_code", "race_whispanicorigin"])
    .agg({"births": sum})
)

In [20]:
mmrates = deaths.merge(births, on=["state", "state_code", "race_whispanicorigin"])

In [21]:
mmrates["rate"] = 100000 * (mmrates["deaths"] / mmrates["births"])

In [22]:
mmrates

,state,state_code,race_whispanicorigin,deaths,births,rate
0,Alabama,01,Black,12.0,17746.0,67.620872
1,Arizona,04,0,15.0,2324.0,645.438898
2,California,06,0,13.0,27520.0,47.238372
3,Florida,12,0,11.0,2935.0,374.787053
4,Florida,12,Black,23.0,45585.0,50.455194
5,Florida,12,White,12.0,88080.0,13.623978
6,Georgia,13,Black,18.0,42373.0,42.479881
7,Illinois,17,White,11.0,70550.0,15.591779
8,Indiana,18,White,17.0,56290.0,30.200746
9,Kentucky,21,White,16.0,40726.0,39.286942


In [23]:
import urllib.request, json

# with urllib.request.urlopen("https://raw.githubusercontent.com/hodcroftlab/covariants/master/web/data/perCountryData.json") as url:
#    data = json.loads(url.read().decode())
#    print(data['regions']['distributions']['California'])

with open("data/usa_states_esri_simple copy.json") as data_file:
    data = json.load(data_file)
# pd.read_json("data/usa_states_esri_simple copy.json"[1])

In [24]:
gpd.read_file('data/usa_states_esri.geojson')

ImportError: The 'read_file' function requires the 'pyogrio' or 'fiona' package, but neither is installed or imports correctly.
Importing fiona resulted in: dlopen(/Users/alexleedsmatthews/.local/share/virtualenvs/notebooks-HSWNtBAS/lib/python3.8/site-packages/fiona/ogrext.cpython-38-darwin.so, 0x0002): Library not loaded: /opt/homebrew/opt/gdal/lib/libgdal.30.dylib
  Referenced from: /Users/alexleedsmatthews/.local/share/virtualenvs/notebooks-HSWNtBAS/lib/python3.8/site-packages/fiona/ogrext.cpython-38-darwin.so
  Reason: tried: '/opt/homebrew/opt/gdal/lib/libgdal.30.dylib' (no such file), '/usr/local/lib/libgdal.30.dylib' (no such file), '/usr/lib/libgdal.30.dylib' (no such file), '/opt/homebrew/Cellar/gdal/3.5.1_2/lib/libgdal.30.dylib' (no such file), '/usr/local/lib/libgdal.30.dylib' (no such file), '/usr/lib/libgdal.30.dylib' (no such file)
Importing pyogrio resulted in: No module named 'pyogrio'

In [ ]:
df = pd.json_normalize(
    pd.read_json("data/usa_states_esri_simple copy.json")["features"]
)

In [ ]:
df.columns = df.columns.str.replace("properties.", "", regex=False).str.lower()

In [ ]:
df.rename(columns={"geometry.coordinates": "geometry"}, inplace=True)

In [ ]:
from shapely.wkt import loads
df.geometry =  df['geometry'].apply(loads)

In [ ]:
df_slim = gpd.GeoDataFrame(
    df[
        [
            "id",
            "geometry",
            "state_name",
            "state_fips",
            "sub_region",
            "state_abbr",
            "population",
            "pop_sqmi",
        ]
    ],
geometry='geometry').copy()

In [ ]:
df_slim.geometry.to_crs('espg:4326')

In [ ]:
df_slim.dtypes

In [ ]:
from vega_datasets import data

In [ ]:
states = alt.topo_feature(data.us_10m.url, feature="states")

In [ ]:
gpd.read_file(data.us_10m.url)

In [ ]:
states

In [ ]:
# background=alt.Chart(states).mark_geoshape(fill="white", stroke="gray").properties(
#    width=300, height=300
# ).project("albersUsa")


states = alt.topo_feature(data.us_10m.url, "states")
source = mmrates

alt.Chart(source).mark_geoshape().encode(
    shape="geo:G",
    color="rate:Q",
    tooltip=["state:N", "rate:Q"],
    facet=alt.Facet("race_whispanicorigin:N", columns=2),
).transform_lookup(
    lookup="id", from_=alt.LookupData(data=states, key="id"), as_="geo"
).properties(
    width=300,
    height=175,
).project(
    type="albersUsa"
)

In [ ]:
import geopandas